In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

In [2]:
df = pd.read_csv("diabetes.csv")
df.head(4)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0


In [3]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [4]:
# sample model  
X=df.iloc[:,:-1].values
y=df['Outcome'].values

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [6]:
X = scaler.fit_transform(X)

In [7]:
X.shape

(768, 8)

In [8]:
from sklearn.model_selection import train_test_split 
X_train , X_test ,y_train , y_test = train_test_split(X,y,random_state=42,test_size=0.2)

In [9]:
import tensorflow 
from tensorflow import keras 
from keras import Sequential 
from keras.layers import Dense 


In [10]:
model = Sequential()
model.add(Dense(32,activation="relu",input_dim=8))
model.add(Dense(1,activation="sigmoid"))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

C:\Users\raj\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
model.fit(X_train,y_train,batch_size=32,epochs=80,validation_data=(X_test,y_test))

Epoch 1/80
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 35ms/step - accuracy: 0.6641 - loss: 0.6725 - val_accuracy: 0.6623 - val_loss: 0.6180
Epoch 2/80
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7026 - loss: 0.6094 - val_accuracy: 0.7078 - val_loss: 0.5782
Epoch 3/80
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6958 - loss: 0.5777 - val_accuracy: 0.7273 - val_loss: 0.5520
Epoch 4/80
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7189 - loss: 0.5624 - val_accuracy: 0.7532 - val_loss: 0.5321
Epoch 5/80
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7317 - loss: 0.5391 - val_accuracy: 0.7532 - val_loss: 0.5190
Epoch 6/80
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7631 - loss: 0.5058 - val_accuracy: 0.7792 - val_loss: 0.5105
Epoch 7/80
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7740 - loss: 0.4904 - val_accuracy: 0.7662 - val_loss: 0.5054
Epoch 8/80
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7644 - loss: 0.5014 - val_accuracy: 0.7727 - val_loss

In [13]:
# how to select the aproperty optimerzer 
# number of nurelas 
# number of hiddent 
# final model with all code 



In [14]:
! pip install keras-tuner --upgrade


In [15]:
import keras_tuner as kt 

In [16]:
def build_model(hp): 
    model = Sequential()
    model.add(Dense(32,activation="relu",input_dim=8))
    model.add(Dense(1,activation="sigmoid"))
    optimizer=['adam','sgd','rmsprop','adadelta']
    model.compile(optimizer=hp.Choice('optimizer',values=optimizer),loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [17]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy' , max_trials=3)

Reloading Tuner from .\untitled_project\tuner0.json


In [18]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [19]:
best_model = tuner.get_best_hyperparameters()
best_model[0].values

{'optimizer': 'rmsprop'}

In [20]:
# model train from 
model = tuner.get_best_models(num_models=1)[0]

C:\Users\raj\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\raj\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [21]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - accuracy: 0.7242 - loss: 0.5147 - val_accuracy: 0.7468 - val_loss: 0.5271
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7300 - loss: 0.5057 - val_accuracy: 0.7468 - val_loss: 0.5230
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7855 - loss: 0.4700 - val_accuracy: 0.7468 - val_loss: 0.5210
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7435 - loss: 0.4906 - val_accuracy: 0.7338 - val_loss: 0.5196
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7394 - loss: 0.4992 - val_accuracy: 0.7532 - val_loss: 0.5188
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7890 - loss: 0.4286 - val_accuracy: 0.7468 - val_loss: 0.5195
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7568 - loss: 0.4747 - val_accuracy: 0.7468 - val_loss: 0.5188
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7684 - loss: 0.4669 - val_accuracy: 0.7

In [33]:
def build_model(hp): 
    model =Sequential()
    untis = hp.Int('units',8,128,step=8)
    model.add(Dense(units=untis,activation="relu",input_dim=8))
    model.add(Dense(1,activation='sigmoid')) 
    model.compile(optimizer="rmsprop",loss='binary_crossentropy',metrics=['accuracy'])
    return model 

In [34]:
tunner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5 , directory='mydir')

Reloading Tuner from mydir\untitled_project\tuner0.json


In [35]:
tunner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [36]:
k=tunner.get_best_hyperparameters()[0].values

In [37]:
k

{'units': 56}

In [38]:
# 56 nerons are best 

In [39]:
model = tunner.get_best_models(num_models=1)[0]

In [40]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 56)                  │             504 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              57 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 561 (2.19 KB)

 Trainable params: 561 (2.19 KB)

 Non-trainable params: 0 (0.00 B)

In [41]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - accuracy: 0.7551 - loss: 0.5201 - val_accuracy: 0.7857 - val_loss: 0.5092
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7722 - loss: 0.4986 - val_accuracy: 0.7857 - val_loss: 0.4987
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7646 - loss: 0.4907 - val_accuracy: 0.7922 - val_loss: 0.4927
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7727 - loss: 0.4797 - val_accuracy: 0.7857 - val_loss: 0.4899
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7714 - loss: 0.4627 - val_accuracy: 0.7987 - val_loss: 0.4885
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7852 - loss: 0.4592 - val_accuracy: 0.7922 - val_loss: 0.4878
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8047 - loss: 0.4416 - val_accuracy: 0.7857 - val_loss: 0.4891
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7802 - loss: 0.4380 - val_accuracy: 0.78

In [46]:
# how to select the number of layers 

def build_model(hp): 
    model =Sequential()
    model.add(Dense(72,activation='relu',input_dim=8))
    for i  in range(hp.Int('num_layers',min_value=1,max_value=10)) : 
        model.add(Dense(1,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))

    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    return model 

In [47]:
tunner = kt.RandomSearch(build_model,objective="val_accuracy",max_trials=3,directory="mydir2",project_name="project")

Reloading Tuner from mydir2\project\tuner0.json


In [48]:
tunner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 15s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 00m 40s


In [49]:
k=tunner.get_best_hyperparameters()[0]
k.values

{'num_layers': 10}

In [50]:
model = tuner.get_best_models(num_models=1)[0]

C:\Users\raj\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\raj\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [51]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [52]:
model.fit(X_train,y_train,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 46ms/step - accuracy: 0.7425 - loss: 0.5220 - val_accuracy: 0.7403 - val_loss: 0.5278
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7820 - loss: 0.4866 - val_accuracy: 0.7403 - val_loss: 0.5235
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7396 - loss: 0.5157 - val_accuracy: 0.7468 - val_loss: 0.5208
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7750 - loss: 0.4733 - val_accuracy: 0.7403 - val_loss: 0.5198
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7822 - loss: 0.4629 - val_accuracy: 0.7532 - val_loss: 0.5185
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8129 - loss: 0.4158 - val_accuracy: 0.7468 - val_loss: 0.5191
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7771 - loss: 0.4482 - val_accuracy: 0.7403 - val_loss: 0.5197
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7767 - loss: 0.4482 - val_accuracy: 0.

In [75]:
def build_model(hp): 
    model = Sequential()
    counter=0 
    
    
    for i in range(hp.Int('num_layers',min_value=1,max_value=10)): 
        
        if counter ==0 : 
            
            model.add(Dense(hp.Int('units'+str(i),min_value=8 , max_value=128 ,step = 8),
                            activation=hp.Choice('activation'+str(i) , values=['relu','tanh','sigmoid'])
                           
                     ,input_dim=8))

        else : 
             model.add(
                     Dense(hp.Int('units'+str(i),min_value=8 , max_value=128 ,step = 8),
                            activation=hp.Choice('activation'+str(i) , values=['relu','tanh','sigmoid'])
                            )
                     )
        counter+=1 
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','nadam']),loss='binary_crossentropy',metrics=['accuracy'])     
    return model

In [76]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=1,  # Fixed argument name
    directory='mydr3',
    project_name='final'
)

C:\Users\raj\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [63]:
# def build_model(hp): 
#     model = Sequential()
#     counter = 0 

#     for i in range(hp.Int('num_layers', min_value=1, max_value=10)): 
        
#         if counter == 0: 
#             model.add(Dense(
#                 hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
#                 activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid']),
#                 input_dim=8  # Should be inside Dense()
#             ))
#         else: 
#             model.add(Dense(
#                 hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
#                 activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid'])
#             ))
        
#         counter += 1  # Corrected counter increment
    
#     model.add(Dense(1, activation='sigmoid'))  # Fixed 'mode.add()' typo

#     model.compile(
#         optimizer=hp.Choice('optimizer', values=['rmsprop', 'adam', 'sgd', 'nadam']),
#         loss='binary_crossentropy',
#         metrics=['accuracy']
#     )     

#     return model

In [77]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 1 Complete [00h 00m 21s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 21s


In [78]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 7,
 'units0': 80,
 'activation0': 'tanh',
 'optimizer': 'nadam',
 'units1': 8,
 'activation1': 'relu',
 'units2': 8,
 'activation2': 'relu',
 'units3': 8,
 'activation3': 'relu',
 'units4': 8,
 'activation4': 'relu',
 'units5': 8,
 'activation5': 'relu',
 'units6': 8,
 'activation6': 'relu'}